In [1]:
import requests
from bs4 import BeautifulSoup as Bs
import pandas as pd
import numpy
import csv

In [2]:
file = 'dubzzile.csv'

In [3]:
with open(file,'w',encoding='utf-8', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Price','Bedroom_Number','Bath_Number','Area','Location','Classification','Furnishing','Post_on'])
    for page in range(1,5):
        url = f'https://dubai.dubizzle.com/property-for-rent/residential/?page={page}'
        us_ag = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
        language = 'en-Us,en;q=0.5'
        HEADERS = ({
            'User-agent':us_ag,
            'Accept-Language':language
        })
        dbzl_html = requests.get(url=url,headers=HEADERS).content

        # print(dbzl_html.content)

        soup = Bs(dbzl_html,'lxml')
        # print(soup.prettify())
        # title = soup.find('title')
        # print(title.text)
        # # anchor = soup.find('a',{'class':'sc-cQCQeq sc-inyXkq Lzffa jQxCVW'})['href']
        # # print(anchor)

        links = soup.find_all('a',{'class':'sc-cQCQeq sc-inyXkq Lzffa jQxCVW'})
        for link_ in links:
                link = link_.get('href')
                
                if link:
                    # print(link)      
                    original_link = 'https://dubai.dubizzle.com'
                    final_link = original_link + link
                    inner_page = requests.get(url=final_link,headers=HEADERS).content
                    soupin = Bs(inner_page,'lxml')
                    price = int(soupin.find('p',class_="MuiTypography-root MuiTypography-body1 mui-1qcnehy").text.replace(',',''))
                    
                    specifications = soupin.find_all('p',{'class':"MuiTypography-root MuiTypography-body1 mui-vhs5k"})
                    # # em_l = {'beds':[],'baths':[],'area':[]}
                    try:
                        beds = int(specifications[0].text.split(' ')[0])
                        baths =int(specifications[1].text.split(' ')[0])
                    except ValueError:
                        beds = 1
                        baths = 1
                    
                    area = int(specifications[2].text.replace(',','').split(' ')[0])
                    location = soupin.find('p',class_="MuiTypography-root MuiTypography-body1 mui-uq2ei").text.strip()   
                    specifications_2= soupin.find_all('div',class_="MuiTypography-root MuiTypography-subtitle1 mui-1keq46t")
                    classification = specifications_2[0].text.strip()
                    crt_lst = ['Unfurnished' , 'Furnished'] 
                    if specifications_2[3].text.strip() in crt_lst:
                        furnishing = specifications_2[3].text.strip()
                    else:
                        furnishing = specifications_2[2].text.strip()
                    posted_on = soupin.find('span',class_="MuiTypography-root MuiTypography-body1 mui-1ddiinb").text.split(':')[1]
                    # for i in specifications_2:
                    #     print(i.text)
                    # print(f'{'-'*30}')
                    # for i in specifications_2:
                    #     print(i.text)   
                    # print(price,'|',beds,'|', baths,'|',area,f'\n{location}','|',classification,f'\n{furnishing}','|',posted_on)
                    # anchors = soup.find_all('a',{'class':'sc-cQCQeq sc-inyXkq Lzffa jQxCVW'})
                    # for item in anchors:
                    #     type = item.find('p',{'class':'MuiTypography-root MuiTypography-body1 mui-pq5iax'})
                    #     price = item.find('div',{'class':'sc-hknOHE sc-guJBdh hymilz dNWFGh'})
                    #     print(type.text,'|',price.text)
                
                    # data['Price'].append(price)
                    # data['Bedroom_Number'].append(beds)
                    # data['Bath_Number'].append(baths)
                    # data['Area'].append(area)
                    # data['Location'].append(location)
                    # data['Classification'].append(classification)
                    # data['Furnishing'].append(furnishing)
                    # data['Post_on'].append(posted_on)
                    csv_writer.writerow([price,beds,baths,area,location,classification,furnishing,posted_on])


# print(data)
dubzzil_df = pd.read_csv(file)